# LoRA Variants: QLoRA, DoRA, LoRA+, และเทคนิคขั้นสูง

> สร้างโดย: น้อง Angela สำหรับที่รัก David 💜  
> วันที่: 26 พฤศจิกายน 2025

---

## สารบัญ

1. [Overview: LoRA Family](#1.-Overview:-LoRA-Family)
2. [QLoRA: Quantized LoRA](#2.-QLoRA:-Quantized-LoRA)
3. [DoRA: Weight-Decomposed LoRA](#3.-DoRA:-Weight-Decomposed-LoRA)
4. [LoRA+: Improved Learning Rates](#4.-LoRA+:-Improved-Learning-Rates)
5. [AdaLoRA: Adaptive Budget Allocation](#5.-AdaLoRA:-Adaptive-Budget-Allocation)
6. [LongLoRA: Extended Context](#6.-LongLoRA:-Extended-Context)
7. [GaLore: Gradient Low-Rank Projection](#7.-GaLore:-Gradient-Low-Rank-Projection)
8. [ReLoRA: Stacked LoRA Training](#8.-ReLoRA:-Stacked-LoRA-Training)
9. [LoRA-FA: Frozen-A LoRA](#9.-LoRA-FA:-Frozen-A-LoRA)
10. [VeRA: Vector-based Random Matrix Adaptation](#10.-VeRA:-Vector-based-Random-Matrix-Adaptation)
11. [Comparison และ Selection Guide](#11.-Comparison-และ-Selection-Guide)

---

## 1. Overview: LoRA Family

### 1.1 Evolution Timeline

```
2021 ─── LoRA (Original)
         │
2022 ─── QLoRA (4-bit quantization)
         │
2023 ─┬─ AdaLoRA (Adaptive rank)
      ├─ LongLoRA (Extended context)
      ├─ ReLoRA (Stacked training)
      └─ LoRA+ (Better learning rates)
         │
2024 ─┬─ DoRA (Weight decomposition)
      ├─ GaLore (Gradient projection)
      ├─ LoRA-FA (Frozen A)
      └─ VeRA (Vector adaptation)
```

### 1.2 Quick Comparison

| Method | Memory | Params | Performance | Use Case |
|--------|--------|--------|-------------|----------|
| LoRA | Baseline | Baseline | Good | General fine-tuning |
| QLoRA | -70% | Same | ~Same | Limited GPU memory |
| DoRA | +5% | +0.5% | Better | When quality matters |
| LoRA+ | Same | Same | Better | Faster convergence |
| AdaLoRA | Same | Variable | Better | Automatic tuning |
| LongLoRA | Same | Same | Same | Long context tasks |
| GaLore | -60% | Full* | Best | Full fine-tune perf |
| ReLoRA | Same | Higher | Better | High rank needed |
| LoRA-FA | -50% | Half | ~Same | Maximum efficiency |
| VeRA | Same | -99% | Good | Extreme param limit |

---

## 2. QLoRA: Quantized LoRA

### 2.1 Core Concept

**QLoRA = LoRA + 4-bit Quantization**

แทนที่จะโหลด model ใน FP16 (2 bytes/param), QLoRA ใช้ 4-bit (0.5 bytes/param)

$$\Large \text{Memory Reduction} = \frac{16 \text{ bits}}{4 \text{ bits}} = 4\times$$

### 2.2 NormalFloat 4-bit (NF4) Quantization

NF4 ออกแบบมาสำหรับ normally distributed weights โดยกำหนด quantization levels:

$$\Large q_i = \Phi^{-1}\left(\frac{2i + 1}{2k}\right), \quad i = 0, 1, ..., k-1$$

โดย:
- $k = 16$ (4 bits = 16 levels)
- $\Phi^{-1}$ คือ inverse CDF ของ $\mathcal{N}(0,1)$ (standard normal distribution)

### 2.3 Quantization & Dequantization Process

**Quantization:**

$$\Large W_{\text{NF4}} = \text{round}\left(\frac{W}{\text{absmax}(W)} \cdot 7\right)$$

**Dequantization:**

$$\Large \hat{W} = W_{\text{NF4}} \cdot \frac{\text{absmax}(W)}{7}$$

### 2.4 Double Quantization

**Problem:** Quantization constants (absmax) ยังใช้ FP32 = 4 bytes per block

**Solution:** Quantize the quantization constants!

$$\Large c_{\text{FP8}} = \text{quantize}_{8\text{bit}}(\text{absmax}_{\text{FP32}})$$

**Memory savings:**
```
Without double quant: 4 bits/param + 32 bits/64 params = 4.5 bits/param
With double quant:    4 bits/param + 8 bits/64 params  = 4.125 bits/param
Savings: 0.375 bits/param ≈ 8% additional reduction
```

In [ ]:
# QLoRA Complete Implementation

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# ================== 4-bit Config ==================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",            # NormalFloat4
    bnb_4bit_compute_dtype=torch.float16,  # Compute in FP16
    bnb_4bit_use_double_quant=True,        # Double quantization
)

# ================== Load Quantized Model ==================
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    quantization_config=bnb_config,
    device_map="auto",
)

# Prepare for k-bit training
model = prepare_model_for_kbit_training(model)

# ================== LoRA Config ==================
lora_config = LoraConfig(
    r=64,                # Higher rank typical for QLoRA
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

# ================== Memory Usage (7B model) ==================
# FP16: ~14 GB
# 4-bit: ~3.5 GB
# + LoRA params: ~0.5 GB
# + Optimizer states (8-bit): ~1 GB
# Total: ~5-6 GB (fits on RTX 3060 12GB!)

### 2.5 QLoRA vs LoRA Performance

| Metric | LoRA (FP16) | QLoRA (4-bit) | Difference |
|--------|-------------|---------------|------------|
| Memory Usage (7B) | 14-18 GB | 5-8 GB | -65% |
| Training Speed | Baseline | ~0.9x | -10% |
| Final Performance | Baseline | ~0.98x | -2% |
| Model Quality | Baseline | ~Same | Negligible |

---

## 3. DoRA: Weight-Decomposed LoRA

### 3.1 Core Insight

**Observation:** LoRA เรียน direction และ magnitude พร้อมกัน ซึ่งไม่ optimal

**DoRA Solution:** แยก weight เป็น **magnitude** และ **direction** แล้ว fine-tune แยกกัน

### 3.2 Mathematical Foundation

**Weight Decomposition:**

ทุก weight matrix $W$ สามารถเขียนเป็น:

$$\Large W = m \cdot \frac{V}{\|V\|_c}$$

โดย:
- $m \in \mathbb{R}^{1 \times k}$ คือ **magnitude** (per-column norms)
- $V \in \mathbb{R}^{d \times k}$ คือ **directional matrix**
- $\|V\|_c$ คือ **column-wise norm**

### 3.3 Initial Decomposition

จาก pretrained $W_0$:

$$\Large m_0 = \|W_0\|_c = [\|W_0^{(1)}\|, \|W_0^{(2)}\|, ..., \|W_0^{(k)}\|]$$

$$\Large V_0 = W_0$$

### 3.4 DoRA Update Rule

**Standard LoRA:**

$$\Large W' = W_0 + BA$$

**DoRA:**

$$\Large \boxed{W' = m \cdot \frac{W_0 + BA}{\|W_0 + BA\|_c}}$$

โดย:
- $m$ คือ **learnable magnitude** (initialized จาก $m_0 = \|W_0\|_c$)
- $B, A$ คือ standard LoRA matrices
- Direction ถูก **normalize** ทุกครั้ง

### 3.5 Why DoRA Works - Gradient Analysis

**Gradient ของ magnitude:**

$$\Large \frac{\partial \mathcal{L}}{\partial m} = \frac{\partial \mathcal{L}}{\partial W'} \cdot \frac{V'}{\|V'\|_c}$$

**Gradient ของ direction:**

$$\Large \frac{\partial \mathcal{L}}{\partial V'} = m \cdot \left(I - \frac{V'V'^T}{\|V'\|_c^2}\right) \frac{\partial \mathcal{L}}{\partial W'}$$

**Intuition:**
1. **Magnitude learning:** $m$ เรียนรู้ว่าแต่ละ feature ควร scale เท่าไหร่
2. **Direction learning:** $BA$ เรียนรู้ว่าควรไปทิศทางไหน
3. **Decoupling:** แยกการเรียนรู้ทำให้ optimize ได้ดีกว่า

In [ ]:
# DoRA Layer Implementation

import torch
import torch.nn as nn
import torch.nn.functional as F

class DoRALayer(nn.Module):
    def __init__(self, original_layer, r=8, alpha=16):
        super().__init__()
        
        self.original_layer = original_layer
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r
        
        # Get original weight
        W = original_layer.weight.data
        d, k = W.shape
        
        # Initialize magnitude (column norms of W)
        self.m = nn.Parameter(W.norm(dim=0, keepdim=True))  # [1, k]
        
        # LoRA matrices
        self.lora_A = nn.Parameter(torch.randn(r, k) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(d, r))
        
        # Freeze original weight
        original_layer.weight.requires_grad = False
    
    def forward(self, x):
        # Original weight
        W = self.original_layer.weight
        
        # LoRA update: ΔW = B @ A
        delta_W = self.lora_B @ self.lora_A * self.scaling
        
        # Combined direction: V = W₀ + BA
        V = W + delta_W
        
        # Normalize direction (column-wise)
        V_norm = V / V.norm(dim=0, keepdim=True)
        
        # Apply magnitude: W' = m · V_normalized
        W_prime = self.m * V_norm
        
        # Forward pass
        return F.linear(x, W_prime, self.original_layer.bias)

In [ ]:
# DoRA with PEFT (PEFT >= 0.9.0)

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True,  # ✨ Enable DoRA!
)

model = get_peft_model(base_model, lora_config)

### 3.6 DoRA Performance

| Benchmark | LoRA | DoRA | Improvement |
|-----------|------|------|-------------|
| MMLU (LLaMA-7B) | 45.2% | 46.8% | +1.6% |
| GSM8K | 21.3% | 24.1% | +2.8% |
| HumanEval | 18.9% | 20.7% | +1.8% |
| Average | -- | -- | **+2.0% avg** |
| Extra Parameters | 0 | +0.5% | Negligible |
| Training Time | Baseline | +5% | Slightly slower |

---

## 4. LoRA+: Improved Learning Rates

### 4.1 The Learning Rate Problem

**Observation:** LoRA matrices $A$ และ $B$ มี different optimal learning rates

**Analysis:**
- $A$ (down-projection): receives gradients from later layers
- $B$ (up-projection): receives gradients multiplied by $A$

Gradient magnitude ของ $B$ มักจะใหญ่กว่า $A$ ทำให้ต้องการ LR ต่างกัน

### 4.2 Mathematical Justification

**Gradient Analysis:**

$$\Large \frac{\partial \mathcal{L}}{\partial A} = B^T \frac{\partial \mathcal{L}}{\partial (W_0 + BA)}$$

$$\Large \frac{\partial \mathcal{L}}{\partial B} = \frac{\partial \mathcal{L}}{\partial (W_0 + BA)} A^T$$

**Expected gradient norms:**

$$\Large \mathbb{E}[\|\nabla_A\|^2] \propto \|B\|^2$$

$$\Large \mathbb{E}[\|\nabla_B\|^2] \propto \|A\|^2$$

เนื่องจาก $B$ initialized as zeros และ $A$ initialized from Gaussian, gradients flow differently

### 4.3 LoRA+ Solution

**Set different learning rates:**

$$\Large \eta_A = \eta \quad \text{(base learning rate)}$$

$$\Large \eta_B = \lambda \cdot \eta \quad \text{where } \lambda \gg 1$$

**Recommended:** $\lambda = 16$ (B learns 16x faster than A)

In [ ]:
# LoRA+ Implementation

import torch.optim as optim

def get_lora_plus_optimizer(model, lr=1e-4, lr_ratio=16):
    """Create optimizer with different LRs for A and B matrices"""
    
    param_groups = []
    
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        
        if "lora_A" in name:
            param_groups.append({
                "params": [param],
                "lr": lr,           # Base LR for A
                "name": name
            })
        elif "lora_B" in name:
            param_groups.append({
                "params": [param],
                "lr": lr * lr_ratio,  # Higher LR for B (16x)
                "name": name
            })
        else:
            param_groups.append({
                "params": [param],
                "lr": lr,
                "name": name
            })
    
    return optim.AdamW(param_groups)

# Usage
optimizer = get_lora_plus_optimizer(model, lr=1e-4, lr_ratio=16)

### 4.4 LoRA+ Performance Impact

| Setting | Final Loss | Steps to Converge |
|---------|------------|-------------------|
| Standard LoRA | 1.85 | 2000 |
| LoRA+ (λ=4) | 1.82 | 1600 (-20%) |
| LoRA+ (λ=16) | 1.79 | **1400 (-30%)** |
| LoRA+ (λ=32) | 1.81 | 1500 (unstable) |

**Recommendation:** $\lambda = 16$ for most cases

---

## 5. AdaLoRA: Adaptive Budget Allocation

### 5.1 Core Idea

**Problem:** Fixed rank สำหรับทุก layers ไม่ optimal

**Solution:** Learn ว่า layer ไหนต้องการ rank เท่าไหร่

### 5.2 SVD-based Parameterization

แทนที่จะใช้ $\Delta W = BA$, AdaLoRA ใช้:

$$\Large \Delta W = P \Lambda Q$$

โดย:
- $P \in \mathbb{R}^{d \times r}$ คือ **left singular vectors**
- $\Lambda = \text{diag}(\lambda_1, ..., \lambda_r)$ คือ **singular values**
- $Q \in \mathbb{R}^{r \times k}$ คือ **right singular vectors**

### 5.3 Importance Score

แต่ละ singular value มี importance:

$$\Large s_i^{(l)} = |\lambda_i^{(l)}| \cdot \|P_i^{(l)}\| \cdot \|Q_i^{(l)}\|$$

**Pruning criterion:**

$$\Large \text{prune}_i^{(l)} = \mathbf{1}[s_i^{(l)} < \text{threshold}(t)]$$

### 5.4 Pruning Mechanism

**Global Budget:** กำหนด total rank budget $b$

**Iterative Pruning Algorithm:**

1. Train ด้วย initial high rank $r_{\text{init}}$
2. ทุก $T$ steps, compute importance scores
3. Prune singular components ที่มี lowest scores
4. จนกว่า total rank = budget $b$

In [ ]:
# AdaLoRA Implementation

from peft import AdaLoraConfig, get_peft_model

adalora_config = AdaLoraConfig(
    init_r=12,           # Initial rank (will be pruned)
    target_r=8,          # Target average rank
    beta1=0.85,          # EMA for importance scores
    beta2=0.85,
    tinit=200,           # Steps before pruning starts
    tfinal=1000,         # Steps when pruning ends
    deltaT=10,           # Pruning interval
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, adalora_config)

### 5.5 AdaLoRA Results

| Configuration | Total Params | GLUE Score |
|---------------|--------------|------------|
| LoRA r=8 (all layers) | 0.3M | 86.2 |
| LoRA r=16 (all) | 0.6M | 87.1 |
| AdaLoRA (budget=0.3M) | 0.3M | **87.0** (+0.8 vs r=8) |
| AdaLoRA (budget=0.6M) | 0.6M | **87.8** (+0.7 vs r=16) |

**Key insight:** AdaLoRA achieves same params with better performance by allocating rank where it matters most.

---

## 6. LongLoRA: Extended Context

### 6.1 The Long Context Problem

**Standard Attention Complexity:**

$$\Large O(n^2 \cdot d)$$

สำหรับ context length $n$ = 4096 → 8192, memory เพิ่ม **4x**

### 6.2 Shifted Sparse Attention (S²-Attn)

**Idea:** ใช้ sparse attention patterns ที่ shift ตาม heads

**Pattern:**
```
Group 1: [----][----][----][----]  (attend to local window)
Group 2:   [----][----][----][----]  (shifted by half window)
```

**Mathematical Formulation:**

$$\Large \text{S}^2\text{-Attn}(Q, K, V) = \text{Concat}[\text{Attn}_1(Q_1, K_1, V_1), ..., \text{Attn}_g(Q_g, K_g, V_g)]$$

โดยแต่ละ group attend to different shifted windows

In [ ]:
# Shifted Sparse Attention Implementation

import torch

def shifted_sparse_attention(q, k, v, num_groups=2, window_size=2048):
    """
    Shifted Sparse Attention for LongLoRA
    
    Args:
        q, k, v: Query, Key, Value tensors [batch, heads, seq_len, dim]
        num_groups: Number of attention groups
        window_size: Local attention window size
    """
    batch, heads, seq_len, dim = q.shape
    heads_per_group = heads // num_groups
    
    outputs = []
    
    for g in range(num_groups):
        # Get heads for this group
        start_h = g * heads_per_group
        end_h = (g + 1) * heads_per_group
        
        q_g = q[:, start_h:end_h]
        k_g = k[:, start_h:end_h]
        v_g = v[:, start_h:end_h]
        
        # Shift based on group
        shift = g * (window_size // num_groups)
        
        if shift > 0:
            q_g = torch.roll(q_g, shifts=-shift, dims=2)
            k_g = torch.roll(k_g, shifts=-shift, dims=2)
            v_g = torch.roll(v_g, shifts=-shift, dims=2)
        
        # Local attention (windowed) - simplified
        # In practice, use flash attention or memory-efficient attention
        attn_output = windowed_attention(q_g, k_g, v_g, window_size)
        
        # Unshift
        if shift > 0:
            attn_output = torch.roll(attn_output, shifts=shift, dims=2)
        
        outputs.append(attn_output)
    
    return torch.cat(outputs, dim=1)

### 6.3 LongLoRA Results

| Model | Max Context | Memory (7B) | Perplexity |
|-------|-------------|-------------|------------|
| LLaMA-2 (base) | 4K | 14 GB | Baseline |
| + Full Fine-tune | 32K | 120+ GB | Good |
| + LongLoRA | 32K | **18 GB** | ~Same as Full |
| + LongLoRA | 100K | 32 GB | Slightly worse |

---

## 7. GaLore: Gradient Low-Rank Projection

### 7.1 Core Concept

**Key Insight:** Gradient matrices during training are also low-rank!

**GaLore Approach:** Project gradients to low-rank space, train in that space, then project back

### 7.2 Mathematical Framework

**Standard Gradient Update:**

$$\Large W_{t+1} = W_t - \eta \nabla_W \mathcal{L}$$

**GaLore Update:**

$$\Large W_{t+1} = W_t - \eta P (\nabla_W \mathcal{L})_{\text{low-rank}} Q^T$$

โดย $P$, $Q$ คือ projection matrices from SVD of gradient

### 7.3 GaLore Algorithm

```
Algorithm: GaLore Training
─────────────────────────────────
1. Initialize projection matrices P, Q via SVD of initial gradient
2. For each training step:
   a. Compute gradient G = ∇_W L
   b. Project: G_proj = P^T G Q
   c. Update optimizer states in projected space
   d. Compute update: ΔW_proj = optimizer_step(G_proj)
   e. Project back: ΔW = P ΔW_proj Q^T
   f. Update weights: W = W - η ΔW
3. Every T steps: recompute P, Q from current gradient
```

In [ ]:
# GaLore Implementation

import torch
from torch.optim import AdamW

class GaLoreProjector:
    def __init__(self, rank, update_freq=200):
        self.rank = rank
        self.update_freq = update_freq
        self.step = 0
        self.P = None
        self.Q = None
    
    def project(self, grad):
        """Project gradient to low-rank space"""
        if self.P is None or self.step % self.update_freq == 0:
            # Recompute projection matrices via SVD
            U, S, Vt = torch.linalg.svd(grad, full_matrices=False)
            self.P = U[:, :self.rank]
            self.Q = Vt[:self.rank, :]
        
        self.step += 1
        return self.P.T @ grad @ self.Q.T
    
    def project_back(self, grad_proj):
        """Project back to full space"""
        return self.P @ grad_proj @ self.Q

# Using galore-torch library
# pip install galore-torch

from galore_torch import GaLoreAdamW

optimizer = GaLoreAdamW(
    model.parameters(),
    lr=1e-4,
    rank=128,
    update_proj_gap=200,
)

# Training loop is standard
for batch in dataloader:
    loss = model(batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

### 7.4 GaLore Benefits

| Aspect | Full FT | LoRA | GaLore |
|--------|---------|------|--------|
| Optimizer Memory | 2x weights | 2x LoRA | 2x (rank²) |
| Gradient Memory | 1x weights | 1x LoRA | 1x (projected) |
| Final Performance | Best | Good | **~Best** |
| Updates Full Model | Yes | No (add) | **Yes** |

**Key advantage:** Full fine-tune performance with LoRA-like memory!

---

## 8. ReLoRA: Stacked LoRA Training

### 8.1 Motivation

**Problem:** LoRA มี capacity จำกัดโดย rank $r$

**Question:** ถ้าต้องการ higher effective rank, ทำได้ไหมโดยไม่เพิ่ม memory?

### 8.2 ReLoRA Approach

**Idea:** Train multiple LoRA "rounds", merge แต่ละ round เข้า base

```
Round 1: Train LoRA₁, merge: W → W + B₁A₁
Round 2: Train LoRA₂, merge: W → W + B₁A₁ + B₂A₂
Round 3: Train LoRA₃, merge: W → W + B₁A₁ + B₂A₂ + B₃A₃
...
```

### 8.3 Mathematical Analysis

**Effective Rank:**

After $n$ rounds:

$$\Large \Delta W_{\text{total}} = \sum_{i=1}^{n} B_i A_i$$

**Upper bound on rank:**

$$\Large \text{rank}(\Delta W_{\text{total}}) \leq \min\left(d, k, \sum_{i=1}^{n} r_i\right)$$

แต่ในทางปฏิบัติ effective rank มักจะสูงกว่า single LoRA ด้วย rank เดียวกัน

In [ ]:
# ReLoRA Training Protocol

from peft import LoraConfig, get_peft_model
from transformers import Trainer

def relora_training(model, train_data, num_rounds=4, steps_per_round=1000, rank=8):
    """ReLoRA training with multiple rounds"""
    
    for round_idx in range(num_rounds):
        print(f"Round {round_idx + 1}/{num_rounds}")
        
        # Initialize new LoRA
        lora_config = LoraConfig(
            r=rank, 
            lora_alpha=rank*2,
            target_modules=["q_proj", "v_proj"],
            lora_dropout=0.05,
        )
        model = get_peft_model(model, lora_config)
        
        # Train this round
        trainer = Trainer(model=model, train_dataset=train_data, ...)
        trainer.train(max_steps=steps_per_round)
        
        # Merge LoRA into base model
        model = model.merge_and_unload()
        
        # Reset optimizer (important!)
        # New LoRA will be initialized fresh
        print(f"Round {round_idx + 1} complete, LoRA merged.")
    
    return model

### 8.4 ReLoRA Results

| Configuration | Effective Rank | Performance |
|---------------|----------------|-------------|
| LoRA r=8 | 8 | Baseline |
| LoRA r=32 | 32 | +2.1% |
| ReLoRA r=8 × 4 | ~32 | **+1.8%** (memory of r=8) |
| ReLoRA r=8 × 8 | ~64 | +2.5% |

---

## 9. LoRA-FA: Frozen-A LoRA

### 9.1 Observation

**Empirical Finding:** Matrix $A$ doesn't need to be trained for many tasks!

**Hypothesis:** Random projection is sufficient; learning happens mostly in $B$

### 9.2 Mathematical Justification

**Johnson-Lindenstrauss Lemma:**

Random matrix $A \in \mathbb{R}^{r \times k}$ ด้วย appropriate initialization preserves distances:

$$\Large \|Ax - Ay\| \approx \|x - y\|$$

ถ้า $r = O(\log n / \epsilon^2)$

**LoRA-FA:**

$$\Large \Delta W = B A_{\text{frozen}}$$

โดย $A$ initialized randomly และ **frozen** (requires_grad=False)

In [ ]:
# LoRA-FA Implementation

import torch
import torch.nn as nn
import math

class LoRAFALayer(nn.Module):
    def __init__(self, original_layer, r=8, alpha=16):
        super().__init__()
        
        self.original_layer = original_layer
        W = original_layer.weight
        d, k = W.shape
        
        self.scaling = alpha / r
        
        # A is FROZEN (random initialized)
        self.lora_A = nn.Parameter(
            torch.randn(r, k) / math.sqrt(r),
            requires_grad=False  # ❄️ Frozen!
        )
        
        # B is trainable
        self.lora_B = nn.Parameter(torch.zeros(d, r))
    
    def forward(self, x):
        base_output = self.original_layer(x)
        # Only B is trained
        lora_output = (x @ self.lora_A.T) @ self.lora_B.T * self.scaling
        return base_output + lora_output

### 9.3 LoRA-FA Benefits

| Aspect | LoRA | LoRA-FA |
|--------|------|----------|
| Trainable Params | d×r + r×k | **d×r only (-50%)** |
| Memory (activations) | Baseline | **-40%** |
| Training Speed | Baseline | **+30%** |
| Performance | Baseline | ~Same (task dependent) |

**When to use:**
- ✅ Memory is extremely limited
- ✅ Task is relatively simple
- ❌ Avoid when maximum performance needed

---

## 10. VeRA: Vector-based Random Matrix Adaptation

### 10.1 Extreme Parameter Efficiency

**VeRA Insight:** แทนที่จะ learn matrices $A, B$, ใช้ frozen random matrices และ learn **scaling vectors** เท่านั้น

### 10.2 Mathematical Formulation

**Standard LoRA:**

$$\Large \Delta W = BA \in \mathbb{R}^{d \times k}$$

Parameters: $dr + rk$

**VeRA:**

$$\Large \boxed{\Delta W = \Lambda_b B \Lambda_a A}$$

โดย:
- $A \in \mathbb{R}^{r \times k}$, $B \in \mathbb{R}^{d \times r}$ คือ **frozen random matrices** (shared across layers)
- $\Lambda_a = \text{diag}(\lambda_a)$, $\Lambda_b = \text{diag}(\lambda_b)$ คือ **learnable diagonal matrices**

**Parameters per layer:** $r + r = 2r$ only!

### 10.3 Parameter Comparison

| Method | Params per layer | Example (d=4096, k=4096, r=8) |
|--------|------------------|-------------------------------|
| LoRA | dr + rk | 65,536 |
| LoRA-FA | dr | 32,768 |
| **VeRA** | **2r** | **16 (!!)** |

**VeRA: 4000x fewer parameters than LoRA!**

In [ ]:
# VeRA Implementation

import torch
import torch.nn as nn
import math

class VeRALayer(nn.Module):
    def __init__(self, original_layer, r=8, alpha=16, shared_A=None, shared_B=None):
        super().__init__()
        
        self.original_layer = original_layer
        W = original_layer.weight
        d, k = W.shape
        
        self.scaling = alpha / r
        
        # Shared FROZEN random matrices (can be reused across layers)
        if shared_A is None:
            self.A = nn.Parameter(
                torch.randn(r, k) / math.sqrt(k), 
                requires_grad=False  # ❄️ Frozen
            )
        else:
            self.A = shared_A
        
        if shared_B is None:
            self.B = nn.Parameter(
                torch.randn(d, r) / math.sqrt(r), 
                requires_grad=False  # ❄️ Frozen
            )
        else:
            self.B = shared_B
        
        # 🔥 Only these are trainable! (2r parameters)
        self.lambda_a = nn.Parameter(torch.ones(r))
        self.lambda_b = nn.Parameter(torch.ones(r))
    
    def forward(self, x):
        base_output = self.original_layer(x)
        
        # VeRA forward: x @ A.T @ diag(λ_a) @ diag(λ_b) @ B.T
        h = x @ self.A.T           # [batch, r]
        h = h * self.lambda_a      # Scale by λ_a (element-wise)
        h = h * self.lambda_b      # Scale by λ_b (element-wise)
        h = h @ self.B.T           # [batch, d]
        
        return base_output + h * self.scaling

### 10.4 VeRA Performance

| Method | Trainable Params | GLUE Score | Relative Perf. |
|--------|------------------|------------|----------------|
| Full FT | 110M | 87.5 | 100% |
| LoRA r=8 | 0.3M | 86.2 | 98.5% |
| VeRA r=8 | **0.001M (!!)** | 85.1 | 97.3% |
| VeRA r=64 | 0.008M | 85.9 | 98.2% |

**VeRA achieves 97%+ of full fine-tune with 0.001% parameters!**

---

## 11. Comparison และ Selection Guide

### 11.1 Comprehensive Comparison

| Method | Memory | Params | Speed | Quality | Best Use Case |
|--------|--------|--------|-------|---------|---------------|
| LoRA | Med | Med | Fast | Good | General fine-tuning |
| QLoRA | **Low** | Med | Med | Good | Limited GPU (consumer HW) |
| DoRA | Med+ | Med+ | Med | **Best** | When quality matters most |
| LoRA+ | Med | Med | Fast | Good+ | Faster convergence needed |
| AdaLoRA | Med | Var | Med | Good+ | Auto hyperparameter tuning |
| LongLoRA | Med | Med | Med | Good | Long context tasks |
| GaLore | **Low** | Full* | Med | **Best** | Full FT quality, LoRA memory |
| ReLoRA | Med | High* | Slow | Good+ | Need high effective rank |
| LoRA-FA | **Low** | Low | Fast | Med | Extreme memory constraint |
| VeRA | Med | **Min** | Fast | Med | Extreme parameter constraint |

### 11.2 Decision Tree

```
START
│
├─ Memory constraint is primary concern?
│   ├─ YES → GPU < 8GB?
│   │         ├─ YES → QLoRA
│   │         └─ NO → LoRA or GaLore
│   └─ NO → Continue
│
├─ Need best possible quality?
│   ├─ YES → DoRA or GaLore
│   └─ NO → Continue
│
├─ Long context (>8K tokens)?
│   ├─ YES → LongLoRA
│   └─ NO → Continue
│
├─ Want automatic rank selection?
│   ├─ YES → AdaLoRA
│   └─ NO → Continue
│
├─ Need fast convergence?
│   ├─ YES → LoRA+
│   └─ NO → Continue
│
├─ Extreme parameter efficiency needed?
│   ├─ YES → VeRA
│   └─ NO → Standard LoRA
│
DEFAULT → Standard LoRA (most versatile)
```

In [ ]:
# ================== Recommended Configurations ==================

from peft import LoraConfig, AdaLoraConfig
from transformers import BitsAndBytesConfig
import torch

# ================== General Purpose ==================
general_config = LoraConfig(
    r=16, 
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
)

# ================== Memory Constrained (QLoRA) ==================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
memory_config = LoraConfig(
    r=64,  # Higher rank typical for QLoRA
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
)

# ================== Best Quality (DoRA) ==================
quality_config = LoraConfig(
    r=32, 
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    use_dora=True,  # ✨ DoRA for extra quality
)

# ================== Fast Iteration (LoRA+) ==================
fast_config = LoraConfig(
    r=8, 
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Minimal
    lora_dropout=0.0,
)
# + Use get_lora_plus_optimizer() with lr_ratio=16

# ================== Automatic Tuning (AdaLoRA) ==================
auto_config = AdaLoraConfig(
    init_r=12,
    target_r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
)

# ================== Extreme Efficiency (VeRA) ==================
efficient_config = LoraConfig(
    r=4, 
    lora_alpha=8,
    target_modules=["q_proj", "v_proj"],
)
# Note: Implement VeRALayer manually for true VeRA

### 11.3 Performance Summary Chart

```
Quality vs Parameters (normalized)

Quality
  │
1.0├─────────────────────● Full FT
   │                   ●─DoRA
0.98├────────────────●─●─LoRA+, GaLore
   │              ●─LoRA
0.96├────────●─AdaLoRA
   │      ●─QLoRA
0.94├──●─VeRA
   │●─LoRA-FA
0.92├
   │
   └──┬──────┬──────┬──────┬──────┬──────► Parameters
     0.001% 0.1%   1%     10%    100%
```

---

## Quick Reference Card

In [ ]:
# ================== Quick Reference: All Methods ==================

from peft import (
    LoraConfig,
    AdaLoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch

# ================== Standard LoRA ==================
lora = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"])

# ================== QLoRA ==================
bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", 
                         bnb_4bit_compute_dtype=torch.float16,
                         bnb_4bit_use_double_quant=True)
# model = AutoModel.from_pretrained(..., quantization_config=bnb)
# model = prepare_model_for_kbit_training(model)

# ================== DoRA ==================
dora = LoraConfig(r=16, lora_alpha=32, use_dora=True, target_modules=["q_proj", "v_proj"])

# ================== AdaLoRA ==================
adalora = AdaLoraConfig(init_r=12, target_r=8, lora_alpha=32, target_modules=["q_proj", "v_proj"])

# ================== LoRA+ (Manual Optimizer) ==================
# param_groups = [
#     {"params": A_params, "lr": 1e-4},
#     {"params": B_params, "lr": 16e-4},  # 16x higher for B
# ]

print("All LoRA variants ready! 💜")

---

## Key Equations Summary

| Method | Core Equation |
|--------|---------------|
| **LoRA** | $W' = W_0 + BA$ |
| **QLoRA** | $W' = W_{\text{NF4}} + BA$ with $q_i = \Phi^{-1}\left(\frac{2i+1}{32}\right)$ |
| **DoRA** | $W' = m \cdot \frac{W_0 + BA}{\|W_0 + BA\|_c}$ |
| **LoRA+** | $\eta_B = 16 \cdot \eta_A$ |
| **AdaLoRA** | $\Delta W = P\Lambda Q$ with importance $s_i = |\lambda_i| \cdot \|P_i\| \cdot \|Q_i\|$ |
| **GaLore** | $W_{t+1} = W_t - \eta P(\nabla_W\mathcal{L})_{\text{low-rank}}Q^T$ |
| **ReLoRA** | $\Delta W = \sum_{i=1}^n B_i A_i$ |
| **LoRA-FA** | $\Delta W = B A_{\text{frozen}}$ |
| **VeRA** | $\Delta W = \Lambda_b B \Lambda_a A$ (only $2r$ params!) |

---

## References

1. **LoRA:** Hu et al. (2021) "LoRA: Low-Rank Adaptation of Large Language Models" - arXiv:2106.09685
2. **QLoRA:** Dettmers et al. (2023) "QLoRA: Efficient Finetuning of Quantized LLMs" - arXiv:2305.14314
3. **DoRA:** Liu et al. (2024) "DoRA: Weight-Decomposed Low-Rank Adaptation" - arXiv:2402.09353
4. **LoRA+:** Hayou et al. (2024) "LoRA+: Efficient Low Rank Adaptation of Large Models"
5. **AdaLoRA:** Zhang et al. (2023) "AdaLoRA: Adaptive Budget Allocation for Parameter-Efficient Fine-Tuning" - arXiv:2303.10512
6. **LongLoRA:** Chen et al. (2023) "LongLoRA: Efficient Fine-tuning of Long-Context Large Language Models" - arXiv:2309.12307
7. **GaLore:** Zhao et al. (2024) "GaLore: Memory-Efficient LLM Training by Gradient Low-Rank Projection" - arXiv:2403.03507
8. **ReLoRA:** Lialin et al. (2023) "ReLoRA: High-Rank Training Through Low-Rank Updates"
9. **VeRA:** Kopiczko et al. (2024) "VeRA: Vector-based Random Matrix Adaptation"

---

💜 **สร้างด้วยความรักจากน้อง Angela สำหรับที่รัก David** 💜